Minor script to prepare catalogs for the voidiness analysis

In [1]:
import pandas as pd
from astropy.cosmology import WMAP9 as cosmo # Used to calculate comoving dist
from footprint_filter import filter_by_footprint

In [2]:
voids = pd.read_excel('exported_dataFrames/voids.xlsx')

In [6]:
def filter_by_redshift(voids_df, data_df):
    too_close = data_df.z < min(voids_df.z)
    data_df = data_df[~too_close]

    too_far = data_df.z > max(voids_df.z)
    return data_df[~too_far]

def add_cmvd(data_df):
    """Adds the comoving distance to data_df"""
    cmvd = cosmo.comoving_distance(data_df['z']) # Comoving distance to void center
    data_df['cmvd_Mpc'] = cmvd.value # add it to data table    
    return data_df

def filter_by_z_ra_dec(data_df_fn, voids, cmvd_add = True, footprint_points_fn = "exported_dataFrames/footprint_points.xlsx"):
    data_df = pd.read_excel(data_df_fn)
    data_df = filter_by_redshift(voids,data_df)
    if cmvd_add:
        data_df = add_cmvd(data_df)
    data_df = filter_by_footprint(data_df, footprint_points_fn)
    return data_df

In [10]:
## 4LAC catalog
four_lac_GRS = filter_by_z_ra_dec('More GRS/FINALCorrectedRedshifts.xlsx', voids)

# Save to excel file
four_lac_GRS.to_excel('exported_dataFrames/z_ra_dec_filtered_4lac.xlsx', index=False)

In [11]:
## xray catalog
xray_agns = filter_by_z_ra_dec('exported_dataFrames/xray_agns.xlsx', voids)
xray_agns.to_excel('exported_dataFrames/z_ra_dec_filtered_xray_agns.xlsx', index = False)